# CRIM Intervals:  Cadences

### What Can You Do With this Notebook?

* Find **cadences**, reporting type, tone, cadential voice functions (roles) and other information about the piece
* Count and analyze distribution of cadences

#### View Score Excerpts with Verovio in the NB


* It is also possible to display the results of the Cadence
    classifier in the Notebook with Verovio. Each excerpt is
    two measures long:  the measure of the final tone of the cadence
    and the previous measure.

* The function also displays metadata about each excerpt, drawn from the
    cadence results dataframe:  piece ID, composer, title, measures, type of
    cadence, beat of the bar in which the final tone is heard, and evaded
    status.
    
* View the cadence table:
   
> `piece.cadences()`
    
* To show the cadences with Verovio, then just:

>`piece.verovioCadences()`

* Note that pink warning messages in the output can be ignored!
   
* If you prefer to create the cadence table, then filter it in some way before passing to Verovio:

>`cadences = piece.cadences()`

(After filtering):

> `piece.verovioCadences(cadences)`




In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
import glob as glob
import os
from IPython.display import SVG

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [2]:
# Select a prefix:
prefix = 'https://crimproject.org/mei/'
prefix = 'Music_Files/'
# just add the CRIM Piece ID here
mei_file = 'in_arboris.mei'
# combine strings and import
url = prefix + mei_file
piece = importScore(url)
print(piece.metadata)

Successfully imported Music_Files/in_arboris.mei
{'title': 'In arboris: A Digital Edition', 'composer': None}


## C. Find Cadences with Modular Analysis

Alex Morgan has built a powerful tool that identifies cadences according to the combinations of two-voice **modules** that describe the typical contrapuntal motion between the various **cadential voice functions** (**CVF**) heard in Renaissance polyphony:  **cantizans and tenorizans**, **cantizans and bassiszans**, etc.

The tool uses modular analysis to identify **conjunctions** of these pairs in order to predict cadences of various kinds. But there are many combinations, especially once we consider that voices functions (or roles) can be **displaced** (as when the tenorizans role appears in the Superius part and the cantizans appears in the Tenor part), or through **irregular** motion, and even **interrupted**, as when a voice is suddenly silent. 

You can in fact check all of the cadential voice functions (CVFs) for a given piece below.  But Alex's system also conveniently **labels** the cadences according to **type**, **tone**, **evaded** and also provides information about the relative place within the piece, the adjacent cadences, and many other features, too.

Note:  **Measure** and **Beat** columns are in the body of the table, not at the Index.

**Column Headings Explained**:

* The **Key** column is the string used by the classifier to determine the label. "BC1" for instance, means "bassus, cantus, and one leading tone".  Note that these letters appear in alphabetical order, not the order of the voices in the score.
* The **CadType** is a high-level label.  **Clausula Vera** is for cadences involving only Cantizans and Tenorizans; **Authentic** is for Cantizans and Bassizans (and possibly the Tenorizans, too).  **Phrygian Clausula Vera** is like **Clausula Vera** but with the half-step motion in the downward-moving (Tenorizans) part.  **Phrygian** corresponds to **Authentic**, except that the Bassizans of course moves up a fifth or down a fourth, as is normally the case when the Tenorizans descends by half=step.  **Altizans Only** is in cases where the Cantizans is missing and the Altizans role moves to a fifth above the lowest voice.  See `print(piece.cvfs.__doc__)` for other labels.
* **Leading Tones** is the count of leading tones motions
* **CVFs** are the **Cadential Voice Functions**, and are listed in order from top to bottom as they appear in the score.  See `print(importedPiece.cvfs.__doc__)` for details.
* The **Low** and **Tone** columns give the pitches of the **lowest sounding pitch (in any voice) at the perfection**, and the **goal tone of the cantizans** (or altizans if there is no cantizans) respectively.
* **RelLow** is the lowest pitch of each cadence shown as an interval measured against the last pitch in the **Low** column. Likewise, **RelTone** is the cadential tone shown as an interval measured against the last pitch in the **Tone** column.
* The **SinceLast** and **ToNext** columns are the time in quarter notes since the last or to the next cadence.
* The **Progress** column is a relative indication of position in the piece.  **0** is the beginning of the piece; **1.0** is the end of the piece.
* **Sounding** is the number of voices heard at the end of the cadence.



Read more via the documentation: **`print(ImportedPiece.cadences.__doc__)`** and especially **print(ImportedPiece.cvfs.__doc__)** for the voice labels

View the **Cadential Voice Function** and **Cadence Label** tables here:  https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/.  These can easily be updated with revised or new cadence types.

### Cadential Voice Functions--What they Mean

In [3]:
print(ImportedPiece.cvfs.__doc__)


        Return a dataframe of cadential voice functions in the piece. If
        `keep_keys` is set to True, the ngrams that triggered each CVF pair
        will be shown in additional columns in the table.

        Each CVF is represented with a single-character label as follows:

        Realized Cadential Voice Functions:
        "C": cantizans motion up a step (can also be ornamented e.g. Landini)
        "T": tenorizans motion down a step (can be ornamented with anticipations)
        "B": bassizans motion up a fourth or down a fifth
        "A": altizans motion, similar to cantizans, but cadences to a fifth
            above a tenorizans instead of an octave
        "L": leaping contratenor motion up an octave at the perfection
        "P": plagal bassizans motion up a fifth or down a fourth
        "Q": quintizans, like a tenorizans, but resolves down by fifth or up by
            fourth to a fourth below the goal tone of a cantizans or an octave
            below the goal tone

### Classify the Cadences

In [5]:

cadences = piece.cadences(keep_keys=True)
col_list = ['Measure', 'Beat', 'CadType', 'Pattern', 'Key', 'Tone','LeadingTones', 'CVFs', 'Low','RelLow','RelTone', 'Sounding', 'Progress','SinceLast','ToNext']
cadences = cadences[col_list]

cadences

,Measure,Beat,CadType,Pattern,Key,Tone,LeadingTones,CVFs,Low,RelLow,RelTone,Sounding,Progress,SinceLast,ToNext


### Summarize by Tone, Type, and Functions

* You can easily summarize the data in various ways: 

> `cadences.groupby(['Tone', 'CadType', 'CVFs']).size().reset_index(name='counts')`

In [17]:
cadences.groupby(['Tone', 'CadType', 'CVFs']).size().reset_index(name='counts')


,Tone,CadType,CVFs,counts
0,A,Phrygian Clausula Vera,CT,4
1,C,Authentic,CB,1
2,D,Altizans Only,AT,1
3,D,Authentic,CB,1
4,D,Authentic,CTB,1
5,D,Clausula Vera,CT,5
6,D,Clausula Vera,CTx,1
7,D,Clausula Vera,CxT,1
8,D,Evaded Authentic,CTb,1
9,D,Phrygian Clausula Vera,CT,4


### C.4.  Display Results with Verovio

* The simplest way to show cadences with Verovio: `piece.verovioCadences()`

But if you prefer to send a curated list of cadences (filtered in some way), then: `piece.cadences()`

* If you prefer to create the cadence table, then filter it in some way before passing to Verovio: `cadences = piece.cadences()`

(After filtering): `piece.verovioCadences(cadences)`

    
* Note that pink warning messages in the output can be ignored!

In [18]:
piece.verovioCadences()

Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 7
Beat:  1.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  TCB


[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 9
Beat:  1.0
Cadence Tone:  D
Cadence Type:  nan
Cadential Voice Functions:  Cu


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 10
Beat:  3.0
Cadence Tone:  D
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 11
Beat:  3.0
Cadence Tone:  D
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


...

Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 122
Beat:  1.0
Cadence Tone:  A
Cadence Type:  Phrygian Clausula Vera
Cadential Voice Functions:  CT


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 126
Beat:  1.0
Cadence Tone:  D
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 128
Beat:  1.0
Cadence Tone:  G
Cadence Type:  Evaded Authentic
Cadential Voice Functions:  Cb


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 130
Beat:  1.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  CuTB


Results:
File Name:  CRIM_Model_0001
Johannes Lupi
Veni speciosam
Cadence End Measure: 132
Beat:  3.0
Cadence Tone:  C
Cadence Type:  Authentic
Cadential Voice Functions:  CB


In [23]:
len(piece.notes().columns)

5